In [5]:
import pandas as pd
import requests
import os
import time
from tqdm import tqdm
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [7]:
# Récupération de la clé depuis l'environnement 
api_key_env = os.environ.get("OMDB_KEY")

In [8]:
API_KEYS = [api_key_env] if api_key_env else []

In [9]:

# --- 1. Configuration (Clé et Limites) ---
DATA_FILE = "titles_netflix.csv"
OUTPUT_FILE = "imdb_scores_export.csv" 


MAX_REQUESTS_PER_KEY = 9000
MAX_REQUESTS = 9000
BASE_URL = "http://www.omdbapi.com/"

# Vérification de la clé API
if not API_KEYS:
    print("ERREUR CRITIQUE: Aucune clé API valide trouvée. Le script ne peut pas s'exécuter.")
    exit() 


# --- 2. Fonction d'Extraction (Titre + Année) - 
def fetch_omdb_data(title, year, api_key):
    """
    Appelle l'API OMDb pour récupérer l'ID IMDb et la Note IMDb.
    """
    params = {
        't': title,
        'y': year, 
        # CORRECTION : 'tomatoes': 'true' est supprimé
        'apikey': api_key
    }
    
    params = {k: v for k, v in params.items() if v} 

    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status() 
        data = response.json()
        
        # Initialisation des résultats (Utilisation de imdb_score)
        results = {
            'title': title, 
            'release_year': year,
            'imdb_id': 'N/A', 
            'imdb_score': 'N/A' 
        }
        
        if data.get('Response') == 'True':
            results['imdb_id'] = data.get('imdbID', 'N/A')
            results['imdb_score'] = data.get('imdbRating', 'N/A')
            
            return results
                
        # Le film n'a pas été trouvé (Response: 'False')
        else:
            return {'title': title, 'release_year': year, 'imdb_id': 'N/A_Not_Found', 'imdb_score': 'N/A_Not_Found'}
            
    except requests.exceptions.RequestException as e:
        return {'title': title, 'release_year': year, 'imdb_id': 'ERROR_API', 'imdb_score': 'ERROR_API'}


# --- 3. Logique de Caching et de Traitement ---

def run_single_phase_process():
    cache_columns = ['title', 'release_year', 'imdb_id', 'imdb_score']
    
    print(f"Démarrage du processus OMDb avec une limite de {MAX_REQUESTS} requêtes/session (avec {len(API_KEYS)} clé active)...")
    
    # Chargement du DataFrame Netflix
    try:
        df_netflix = pd.read_csv(DATA_FILE)
    except FileNotFoundError:
        print(f"ERREUR: Fichier d'entrée non trouvé: {DATA_FILE}. Veuillez vérifier le nom du fichier.")
        return

    # Vérification et Préparation des colonnes
    if 'title' not in df_netflix.columns or 'release_year' not in df_netflix.columns:
        print("ERREUR: Les colonnes 'title' et/ou 'release_year' sont manquantes.")
        return
        
    df_netflix['release_year'] = df_netflix['release_year'].astype(str).str.replace(r'\.0$', '', regex=True)

    # Chargement ou initialisation du Cache 
    if os.path.exists(OUTPUT_FILE):
        try:
            df_cache = pd.read_csv(OUTPUT_FILE)
            if 'imdb_score' in df_cache.columns:
                 # Filtre les lignes déjà traitées avec succès
                 df_cache = df_cache[~df_cache['imdb_score'].isin(['ERROR_API', 'N/A_Not_Found'])]
                 df_cache = df_cache[[c for c in cache_columns if c in df_cache.columns]]
            else:
                 df_cache = pd.DataFrame(columns=cache_columns)
                 
        except Exception:
            df_cache = pd.DataFrame(columns=cache_columns)
            print(" Erreur de lecture du cache, réinitialisation du cache.")
            
        print(f" Cache chargé: {len(df_cache)} titres déjà traités avec succès.")
    else:
        df_cache = pd.DataFrame(columns=cache_columns)
        print("Cache initialisé : Première exécution.")

    # Créer une clé unique pour le filtrage (Titre + Année)
    df_netflix['unique_key'] = df_netflix['title'].str.cat(df_netflix['release_year'], sep='|')
    df_cache['unique_key'] = df_cache['title'].str.cat(df_cache['release_year'].astype(str), sep='|')
    
    # Identifier les titres restants à traiter
    merged_df = pd.merge(df_netflix, df_cache, on='unique_key', how='left', suffixes=('_netf', '_cache'))
    
    # On ne garde que les lignes qui n'ont pas encore de score IMDb (colonne créée par le merge)
    df_to_process = merged_df[merged_df['imdb_score'].isna()] 
    
    print(f"Titres restants à traiter: {len(df_to_process)}")

    # Appliquer la limite de la session
    df_to_process_session = df_to_process.head(MAX_REQUESTS)
    print(f"Début du traitement de {len(df_to_process_session)} titres pour cette session.")

    new_results = []
    processed_count = 0
    
    # Boucle de traitement
    for index, row in tqdm(df_to_process_session.iterrows(), total=len(df_to_process_session)):
        
        # NOTE : Il n'y a plus de rotation, on utilise la seule clé valide [0]
        current_api_key = API_KEYS[0] 
        
        result = fetch_omdb_data(row['title_netf'], row['release_year_netf'], current_api_key)
        
        new_results.append(result)
        processed_count += 1
        
        time.sleep(0.5) # Pause de sécurité

    # 4. Enregistrer les résultats (Mise à jour du Cache)
    if new_results:
        df_new_results = pd.DataFrame(new_results)
        
        df_combined = pd.concat([df_cache.drop(columns='unique_key', errors='ignore'), df_new_results])
        df_combined = df_combined.drop_duplicates(subset=['title', 'release_year'], keep='last')
        
        df_combined.to_csv(OUTPUT_FILE, index=False)
        
        print("\n--- Session Terminée ---")
        print(f"{processed_count} nouvelles lignes traitées.")
        print(f"Fichier de résultat (cache) enregistré : {OUTPUT_FILE}")
    else:
        print("Aucun nouveau titre traité. Le cache est à jour pour cette limite.")


# --- Lancement du Processus ---
if __name__ == "__main__":
    run_single_phase_process()

Démarrage du processus OMDb avec une limite de 9000 requêtes/session (avec 1 clé active)...
Cache initialisé : Première exécution.
Titres restants à traiter: 8793
Début du traitement de 8793 titres pour cette session.


100%|██████████| 8793/8793 [2:18:41<00:00,  1.06it/s]     


--- Session Terminée ---
8793 nouvelles lignes traitées.
Fichier de résultat (cache) enregistré : imdb_scores_export.csv
